In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from resnet import ResNet18, ResNet34
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

MAX_TRAINING = 5000
MAX_VALIDATION = 2000
MAX_TESTING = 2000
correct_class = {1 : 'airplane', 2 : 'automobile', 3 : 'bird', 4 : 'cat', 5 : \
                 'deer', 6 : 'dog', 7 : 'frog', 8 : 'horse', 9 : 'ship', 10 : 'truck'}

#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (x_test, y_test) = cifar.load_data()

x_train_max, y_train_max = get_random_data(x_train, y_train, 0, MAX_TRAINING*3, MAX_TRAINING)
x_val, y_val = get_random_data(x_train, y_train, MAX_TRAINING*3+1, x_train.shape[0], MAX_VALIDATION)
x_test_max, y_test_max = get_random_data(x_test, y_test, 0, x_test.shape[0], MAX_TESTING)

# Keeping the data between 0 and 1
x_train, x_test = x_train_max / 255.0, x_test_max / 255.0
y_train = to_categorical(y_train_max, num_classes=10)
y_test = to_categorical(y_test_max, num_classes=10)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=5000, Y=5000
Number of Validation Samples: X=2000, Y=2000
Number of Test Samples: X=2000, Y=2000


In [3]:
class MetricsAfterEachEpoch(tf.keras.callbacks.Callback):
    def on_train_begin(self, scores={}):
        self.loss, self.accuracy, self.mae, self.mse = [], [], [], []
    def on_train_end(self, logs={}):
        return self.loss, self.accuracy, self.mae, self.loss
    def on_epoch_end(self, epoch, scores):
        self.loss.append(scores.get('loss'))
        self.accuracy.append(scores.get('acc'))
        self.mae.append(scores.get('mean_absolute_error'))
        self.loss.append(scores.get('mean_squared_error'))
        return

In [4]:
def ResNetEvaluator(curr_optimizer, x_train, y_test, label=None, curr_epochs=25):
    img_rows, img_cols, img_channels = x_train[0].shape
    nb_classes = y_test[0].shape[0]
    
    model = ResNet34((img_rows, img_cols, img_channels), nb_classes)
    
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, callbacks=[MetricsAfterEachEpoch()], batch_size=2048).history
    score = model.evaluate(x_test, y_test, batch_size=512)
    #Saving the model
    if label == 'RAdam':
        model.save("../saved_models/{}_{}_{}.hdf5".format('RAdam', 'relu', "ResNet34"))
    else: 
        model.save("../saved_models/{}_{}_{}.hdf5".format(curr_optimizer, 'relu', "ResNet34"))
    return score, history

In [5]:
optimizer = 'Adam'
score_returned, history = ResNetEvaluator(optimizer, x_train, y_test, 'Adam', 10)

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 30s 6ms/sample - loss: 4.1150 - accuracy: 0.2112 - mae: 0.1654 - mse: 0.0929
Epoch 2/10
5000/5000 [==============================] - 33s 7ms/sample - loss: 3.0510 - accuracy: 0.4766 - mae: 0.1334 - mse: 0.0671
Epoch 3/10
5000/5000 [==============================] - 26s 5ms/sample - loss: 2.5149 - accuracy: 0.6220 - mae: 0.1107 - mse: 0.0518
Epoch 4/10
5000/5000 [==============================] - 185s 37ms/sample - loss: 2.0600 - accuracy: 0.7510 - mae: 0.0868 - mse: 0.0372
Epoch 5/10
5000/5000 [==============================] - 93s 19ms/sample - loss: 1.6587 - accuracy: 0.8672 - mae: 0.0617 - mse: 0.0226
Epoch 6/10
5000/5000 [==============================] - 200s 40ms/sample - loss: 1.3490 - accuracy: 0.9390 - mae: 0.0391 - mse: 0.0119
Epoch 7/10
5000/5000 [=============================

In [6]:
optimizer = 'Adam'
score_returned, history = ResNetEvaluator(RAdam(lr=0.005), x_train, y_test, 'RAdam', 8)

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Train on 5000 samples
Epoch 1/8
5000/5000 [==============================] - 141s 28ms/sample - loss: 4.4521 - accuracy: 0.1048 - mae: 0.1797 - mse: 0.0985
Epoch 2/8
5000/5000 [==============================] - 129s 26ms/sample - loss: 4.3243 - accuracy: 0.1244 - mae: 0.1780 - mse: 0.0961
Epoch 3/8
5000/5000 [==============================] - 29s 6ms/sample - loss: 3.8513 - accuracy: 0.2412 - mae: 0.1666 - mse: 0.0860
Epoch 4/8
5000/5000 [==============================] - 61s 12ms/sample - loss: 3.3866 - accuracy: 0.4108 - mae: 0.1472 - mse: 0.0737
Epoch 5/8
5000/5000 [==============================] - 132s 26ms/sample - loss: 3.0548 - accuracy: 0.5148 - mae: 0.1324 - mse: 0.0638
Epoch 6/8
5000/5000 [==============================] - 97s 19ms/sample - loss: 2.7373 - accuracy: 0.6166 - mae: 0.1190 - mse: 0.0536
Epoch 7/8
5000/5000 [==============================] -